# Day 7: Demand I - Enhancing the model with demand

We will continue our modeling exercise by including demand responses.

This will allow us to consider the policy impacts of alternative energy transition policies, when consumers face different pricing regimes.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [ ]:
#using Pkg
#Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Cbc", "Plots", "Printf"])

In [1]:
 
using DataFrames
using CSV
using JuMP
using Ipopt, Cbc
using Plots
using Printf


## Building the model

We load the same data as usual, and also clean it up to simplify it further and create the demand and import curves.

In [2]:

dfclust = CSV.read("data_jaere_clustered.csv", DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate imports (using elas 0.3)
dfclust.bm = 0.3 * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept
    
describe(dfclust)



,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,price,38.5055,1.66311,37.2797,137.116,0,Float64
2,imports,7.52426,4.2479,7.57055,9.79814,0,Float64
3,q_commercial,12.9532,9.26236,12.3881,22.1766,0,Float64
4,q_industrial,4.1224,2.60049,3.83347,7.62003,0,Float64
5,q_residential,11.0753,4.3995,10.0075,20.4922,0,Float64
6,wind_cap,0.340955,0.0935896,0.334676,0.681777,0,Float64
7,solar_cap,0.260549,0.00155919,0.0679638,0.793407,0,Float64
8,hydronuc,5.0346,2.36316,4.56859,9.68051,0,Float64
9,weights,0.0876,0.00787044,0.0788053,0.227839,0,Float64


We also load the technology file

In [3]:

tech = CSV.read("data_technology.csv", DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;
tech.F2 = tech.F2 ./afactor;
tech


,techname,heatrate,heatrate2,F,capLB,capUB,new,renewable
,String15,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,Hydro/Nuclear,10.0,0.0,0.0,1.0,1.0,0,0
2,Existing 1,6.67199,0.0929123,0.0,11.5,11.5,0,0
3,Existing 2,9.79412,0.286247,0.0,14.5,14.5,0,0
4,Existing 3,13.8181,20.5352,0.0,0.578,0.578,0,0
5,New Gas,6.6,0.0,78.4773,0.0,100.0,1,0
6,Wind,0.0,0.0,100.303,0.0,100.0,1,1
7,Solar,0.0,0.0,100.303,0.0,100.0,1,1


### Adding demand considerations

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$$ \max \ CS - Costs - Fixed Costs \\

\text{s.t.} \ \text{operational constraints, market clearing}. $$

To include demand, we will separate the price that consumers pay from the price that the wholesale market sets. We define a variable "tariff" that is what consumers pay any hour of the day.

We also allow alpha to determine the share of "insensitive" consumers. We can see how the investment changes as we modify "alpha".

In [10]:
function clear_market_invest_tariff(data::DataFrame, tech::DataFrame;
    subsidy=0.0, tax=0.0, elas = 0.3,tariff= 40.0,alpha=1.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

     # Setting demand parameters
     data.b = elas * data.demand ./ data.price;  # slope
     data.a = data.demand + data.b .* data.price;  # intercept
    
    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, total_cost);
    @variable(model, profit[1:I]);  # profits by technology need to be at least zero
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * (alpha*tariff + (1.0-alpha)*price[t]));
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
        # insenstive consumers
                alpha * (tariff * (data.a[t] - data.b[t]*tariff) 
            + (data.a[t] - data.b[t]*tariff)^2/(2*data.b[t])) 
        # sensitive consumers
            + (1.0-alpha)*(price[t] * (data.a[t] - data.b[t]*price[t]) 
            + (data.a[t] - data.b[t]*price[t])^2/(2*data.b[t])));

 # Define costs
@constraint(model, [t=1:T], costs[t] ==
        sum(tech.c[i] * quantity[t,i]
        + tech.c2[i] * quantity[t,i]^2/2 
        + tax * tech.e[i] * quantity[t,i] 
        - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
+ (imports[t] - data.am[t])^2/(2 * data.bm[t]))


@constraint(model, total_cost == sum(data.weights[t] * costs[t] for t=1:T)
 + tech.F[5]*gas_gw  + tech.F[6]*wind_gw + tech.F[7]*solar_gw 
 + tech.F2[5]*gas_gw^2  + tech.F2[6]*wind_gw^2 + tech.F2[7]*solar_gw^2)

# Define profits
@constraint(model, [i=1:4], profit[i] == 
        sum(data.weights[t]*(price[t] 
    - tech.c[i] - tech.c2[i]*quantity[t,i]/2)*quantity[t,i] for t=1:T));
@constraint(model, profit[5] == 
        sum(data.weights[t]*(price[t] 
    - tech.c[5] - tech.c2[5]*quantity[t,5]/2)*quantity[t,5] for t=1:T)
    - tech.F[5]*gas_gw - tech.F2[5]*gas_gw^2);
@constraint(model, profit[6] == 
        sum(data.weights[t]*(price[t] 
    - tech.c[6] - tech.c2[6]*quantity[t,6]/2)*quantity[t,6] for t=1:T)
    - tech.F[6]*wind_gw - tech.F2[6]*wind_gw^2);
@constraint(model, [i=1:4], profit[7] == 
        sum(data.weights[t]*(price[t] 
    - tech.c[7] - tech.c2[7]*quantity[t,7]/2)*quantity[t,7] for t=1:T)
    - tech.F[7]*solar_gw - tech.F2[7]*solar_gw^2);



        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        d = JuMP.value.(demand);
        q = JuMP.value.(quantity);
        d_insen = (data.a .- data.b.*tariff);
        d_sen = (data.a .- data.b.*p);
        avg_price = sum(p.*d.*data.weights)/sum(d.*data.weights);
        avg_price_insen = sum(p.*d_insen.*data.weights)/sum(d_insen.*data.weights);
        avg_price_sen = sum(p.*d_sen.*data.weights)/sum(d_sen.*data.weights);
        imp = JuMP.value.(imports);
        cost = JuMP.value.(costs);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_price_insen" => avg_price_insen,
            "avg_price_sen" => avg_price_sen,
            "price" => p,
            "quantity" => q,
            "quantity_sen" => d_sen,
            "quantity_insen" => d_insen,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "total_cost" => JuMP.value.(total_cost),
            "profits" => JuMP.value.(profit),
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        @printf("Solving error!")
        return results
    end

end

clear_market_invest_tariff (generic function with 1 method)

We will set the initial tariff to $30/MWh.

In [11]:
results1 = clear_market_invest_tariff(dfclust, tech)

Dict{String, Any} with 16 entries:
  "avg_price"       => 42.6059
  "price"           => [42.6171, 35.1464, 26.225, 38.9551, 27.0243, 26.8164, 26…
  "gas_gw"          => 1.87949
  "quantity_sen"    => [39.4012, 27.6118, 28.851, 28.957, 23.9866, 26.467, 27.4…
  "status"          => "LOCALLY_SOLVED"
  "quantity"        => [9.46003 11.5 … 6.82405e-9 3.49202e-9; 4.0713 11.5 … 7.4…
  "solar_gw"        => 0.0
  "imports"         => [8.79488, 8.13045, 7.93675, 6.94785, 5.66144, 7.04172, 7…
  "avg_price_sen"   => 16.0846
  "demand"          => [39.9566, 26.4467, 25.3595, 28.683, 21.6728, 22.5578, 24…
  "avg_price_insen" => 42.6059
  "cost"            => [797.333, 448.557, 357.405, 586.45, 375.432, 369.875, 38…
  "quantity_insen"  => [39.9566, 26.4467, 25.3595, 28.683, 21.6728, 22.5578, 24…
  "total_cost"      => 4530.39
  "profits"         => [931.943, 862.347, 304.843, 7.3963, 10.379, 3.06807e-6, …
  "wind_gw"         => 0.0

Notice that average weighted prices are higher than the tariff we charged consumers, we need to charge them more or otherwise, we would not be able to pay the producers of energy.

In [12]:
results2 = clear_market_invest_tariff(dfclust, tech, tariff=50.0)

Dict{String, Any} with 16 entries:
  "avg_price"       => 46.7856
  "price"           => [41.9624, 33.6989, 25.8759, 37.8299, 26.9125, 26.3308, 2…
  "gas_gw"          => 0.445763
  "quantity_sen"    => [39.5402, 27.9593, 28.9395, 29.2521, 24.0066, 26.611, 27…
  "status"          => "LOCALLY_SOLVED"
  "quantity"        => [9.46003 11.5 … 6.27049e-9 2.47795e-9; 4.0713 11.5 … 7.3…
  "solar_gw"        => 0.0
  "imports"         => [8.7601, 8.029, 7.90949, 6.88237, 5.65589, 7.00344, 7.82…
  "avg_price_sen"   => 5.16896
  "demand"          => [37.8346, 24.0461, 22.8249, 26.0606, 19.8897, 19.5926, 2…
  "avg_price_insen" => 46.7856
  "cost"            => [734.205, 381.002, 294.704, 506.793, 331.985, 295.156, 3…
  "quantity_insen"  => [37.8346, 24.0461, 22.8249, 26.0606, 19.8897, 19.5926, 2…
  "total_cost"      => 4256.05
  "profits"         => [879.207, 772.398, 273.363, 6.63368, 0.582942, 2.80832e-…
  "wind_gw"         => 0.0

We can see the right tariff is between \\$30 and \\$50/MWh.

### Solving for the fixed point
We can solve for the fixed point in a loop.

In [13]:

current_diff = 1.0;
guess = 40.0;
    while (current_diff > 1e-2)
        res = clear_market_invest_tariff(dfclust, tech, tariff=guess);
        newguess = res["avg_price_insen"];
        current_diff = (guess-newguess).^2;
        guess = newguess;
    end
    guess


44.025259418951734

Note: It is best to convert this into a function so that now we can compare alternative solutions.

In [14]:
function clear_market_equilibrium(data::DataFrame, tech::DataFrame; 
        tariff=40.0, alpha=1.0, tax=0.0, subsidy=0.0, elas = 0.3)
    
    current_diff = 1.0;
    guess = tariff;
    while (current_diff > 1e-3)
        res = clear_market_invest_tariff(dfclust, tech, tariff=guess, alpha=alpha,
            tax=tax, subsidy=subsidy,  elas=elas);
        newguess = res["avg_price_insen"];
        current_diff = (guess-newguess).^2;
        guess = newguess ;
    end
    
    # we solve at the equilibrium guess and save tariff
    res = clear_market_invest_tariff(dfclust, tech, tariff=guess, alpha=alpha,
            tax=tax, subsidy=subsidy);
    res["tariff"] = guess;
    
    return res
    
end

clear_market_equilibrium (generic function with 1 method)

In [15]:
res_sen = clear_market_equilibrium(dfclust, tech, alpha=0.0, tariff=32.0)

Dict{String, Any} with 17 entries:
  "avg_price"       => 34.7697
  "price"           => [41.9254, 35.7935, 27.7567, 38.9213, 34.3886, 34.1268, 3…
  "gas_gw"          => 2.19809
  "quantity_sen"    => [39.548, 27.4565, 28.4628, 28.9659, 22.6734, 24.2993, 25…
  "status"          => "LOCALLY_SOLVED"
  "quantity"        => [9.46003 11.5 … 6.82405e-9 3.49202e-9; 4.0713 11.5 … 7.4…
  "solar_gw"        => 0.0
  "tariff"          => 36.395
  "imports"         => [8.75814, 8.1758, 8.05636, 6.94588, 6.02714, 7.61806, 8.…
  "avg_price_sen"   => 34.7697
  "demand"          => [39.548, 27.4565, 28.4628, 28.9659, 22.6734, 24.2993, 25…
  "avg_price_insen" => 36.3964
  "cost"            => [774.246, 480.725, 439.331, 592.71, 403.664, 417.907, 42…
  "quantity_insen"  => [40.7216, 27.3121, 26.2733, 29.6283, 22.3157, 23.6268, 2…
  "total_cost"      => 4377.23
  "profits"         => [925.816, 872.183, 89.4503, 0.296038, 14.3213, 3.00097e-…
  "wind_gw"         => 0.0

In [16]:
res_insen = clear_market_equilibrium(dfclust, tech, alpha=1.0, tariff=43.0)

Dict{String, Any} with 17 entries:
  "avg_price"       => 44.0517
  "price"           => [42.3837, 34.8112, 26.0942, 38.5312, 26.9896, 26.6304, 2…
  "gas_gw"          => 1.26616
  "quantity_sen"    => [39.4508, 27.6923, 28.8841, 29.0682, 23.9928, 26.5221, 2…
  "status"          => "LOCALLY_SOLVED"
  "quantity"        => [9.46003 11.5 … 6.82404e-9 3.49202e-9; 4.0713 11.5 … 7.4…
  "solar_gw"        => 0.0
  "tariff"          => 44.0467
  "imports"         => [8.78248, 8.10696, 7.92654, 6.92318, 5.65972, 7.02706, 7…
  "avg_price_sen"   => 11.9189
  "demand"          => [39.0979, 25.4752, 24.3338, 27.6218, 20.9512, 21.3579, 2…
  "avg_price_insen" => 44.0517
  "cost"            => [772.268, 421.394, 331.982, 554.462, 357.871, 339.563, 3…
  "quantity_insen"  => [39.0979, 25.4752, 24.3338, 27.6218, 20.9512, 21.3579, 2…
  "total_cost"      => 4401.88
  "profits"         => [912.934, 823.292, 294.512, 7.15432, 4.70319, 3.06655e-6…
  "wind_gw"         => 0.0

In [ ]:
histogram(res_sen["price"], weights=dfclust.weights)

## Follow-up exercises

1. What take aways do you get from the above visualization?

2. Consider an energy savings technology. How could it be added to the model? What would be the challenges in calibrating it?
3. Assume that consumers only pay attention to the average price instead of the real wholesale price, although they keep paying the later. Let the average price equal to the tariff. How would you change your model?